# Mumbai

In [ ]:
# import os,sys;sys.path.append(os.path.dirname(os.path.realpath('mail')))
# from mail.send_mail import sendMail
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import DataFrame
import datetime as dt
from datetime import datetime, timedelta,date
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

# try:
#     gc = pygsheets.authorize(service_file='/home/karan/config/car-master-sheet.json')
gc = pygsheets.authorize(service_file='car-master-sheet.json')


# gc = pygsheets.authorize(service_file='/home/ubuntu/scripts/client_secret.json')

today = date.today() + timedelta(days = 1)

allotment_wkbk = gc.open("Allotment Status Report")


allotment_history_sheet = allotment_wkbk.worksheet_by_title("New Allotment History")

allotment_history_data = pd.DataFrame(allotment_history_sheet.get_all_records())

allotment_history_data = allotment_history_data[['Car + Count','Car Number','Allotment Date','ETM','Return Date']]

allotment_history_data['Return Date'] = allotment_history_data['Return Date'].replace(r'^\s*$', today, regex=True)

allotment_history_data = allotment_history_data.replace(r'^\s*$', np.nan, regex=True)

allotment_history_data = allotment_history_data[allotment_history_data['ETM'].str.contains('ET',na=False)]

allotment_history_data['Allotment Date'] = pd.to_datetime(allotment_history_data['Allotment Date'], dayfirst = True)

allotment_history_data['Return Date'] = pd.to_datetime(allotment_history_data['Return Date'], dayfirst = True)


allotment_history_data['Allotment Date Dummy'] = allotment_history_data['Allotment Date'].dt.date

allotment_history_data['Return Date Dummy'] = allotment_history_data['Return Date'].dt.date

adjustment_given_allotment_history = allotment_history_data[['Car Number','Return Date Dummy','Return Date','ETM']]

adjustment_taken_allotment_history = allotment_history_data[['Car + Count','ETM','Allotment Date Dummy','Allotment Date']]

audit_data = allotment_wkbk.worksheet_by_title("Audit")

allotment_audit_data = pd.DataFrame(audit_data.get_all_records())

allotment_audit_data = allotment_audit_data[['Car + Count','Car Number','CNG - How many bars','Fuel indicator Petrol','Adjustment Amount','Timestamp','DM']]

allotment_audit_data = allotment_audit_data.replace(r'^\s*$', np.nan, regex=True)

allotment_audit_data = allotment_audit_data[allotment_audit_data['Fuel indicator Petrol'].notna()]

allotment_audit_data['Timestamp'] = pd.to_datetime(allotment_audit_data['Timestamp'], dayfirst = True)

# allotment_audit_data['start_date'] = pd.Timestamp('07-06-2022')

allotment_audit_data['start_date'] = pd.Timestamp('08-07-2022')#changes

allotment_audit_data = allotment_audit_data.loc[(allotment_audit_data['Timestamp'] > allotment_audit_data['start_date'])]

allotment_audit_data = allotment_audit_data.loc[~(allotment_audit_data['Adjustment Amount'] == 0)]

allotment_audit_data['Return Date'] = allotment_audit_data['Timestamp']

allotment_audit_data.drop(['start_date'],axis = 1, inplace = True)#changes

adjustment_given = pd.merge(allotment_audit_data,adjustment_given_allotment_history,on=['Return Date','Car Number'],how='inner')

adjustment_given.rename(columns={"Return Date Dummy":"Bill Date"},inplace = True)

adjustment_given.drop(['Car + Count'],axis = 1, inplace = True)

# adjustment_given['start_date'] = pd.Timestamp('08-02-2022')

adjustment_given['start_date'] = pd.Timestamp('08-07-2022')#changes

adjustment_given = adjustment_given.loc[(adjustment_given['Timestamp'] > adjustment_given['start_date'])]

adjustment_given.drop(['start_date'],axis = 1, inplace = True)




allotment_audit_data.drop(['Timestamp'],axis = 1, inplace = True)

adjustment_taken = pd.merge(allotment_audit_data,adjustment_taken_allotment_history,on='Car + Count',how='inner')
adjustment_taken['Adjustment Amount New'] = adjustment_taken['Adjustment Amount'] * -1

adjustment_taken

adjustment_taken.drop(['Car + Count','Adjustment Amount'],axis = 1, inplace = True)

# adjustment_taken.drop(['Adjustment Amount'],axis = 1, inplace = True)


adjustment_taken.rename(columns={"Allotment Date Dummy":"Bill Date","Allotment Date":"Timestamp","Adjustment Amount New":"Adjustment Amount"},inplace = True)




print(adjustment_given.head())
print(adjustment_taken.head())


adjustment_final = pd.concat([adjustment_taken, adjustment_given], ignore_index=True)


adjustment_final['Name of DM'] = 'Allotment'
adjustment_final['Pilot Name'] = ''
adjustment_final['Adjustment Reason'] = 'Excess Petrol/CNG' 
adjustment_final['Adjustment Comment'] = "CNG:" + adjustment_final['CNG - How many bars'].astype(str) + " / Petrol: " + adjustment_final['Fuel indicator Petrol'].astype(str)
adjustment_final.drop(['CNG - How many bars','Fuel indicator Petrol'],axis = 1, inplace = True)

adjustment_final.sort_values(by=['Timestamp'], ascending=True, inplace=True)

adjustment_final


# adjustment_gs = gc.open("New-Adjustment Sheet")

# adjustment_sheet = adjustment_gs.worksheet_by_title("New Allotment Fuel")

# adjustment_sheet.set_dataframe(adjustment_final, 'A1')



# except Exception:
#     sendMail([
#     'abusalameverestfleet@gmail.com',
#     'avinash.everestfleet@gmail.com',
#     'rutvijaevrestfleet@gmail.com',
#     'sameer.everestfleet@gmail.com',
#     'Karan.saraogi@everestfleet.com',
#     'vijugode@gmail.com'
#     ])

# test

In [5]:
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import DataFrame
import datetime as dt
from datetime import datetime, timedelta,date
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

# try:
#     gc = pygsheets.authorize(service_file='/home/karan/config/car-master-sheet.json')
gc = pygsheets.authorize(service_file='car-master-sheet.json')


# gc = pygsheets.authorize(service_file='/home/ubuntu/scripts/client_secret.json')

today = date.today() + timedelta(days = 1)

allotment_wkbk = gc.open("Copy of Allotment Status Report")


allotment_history_sheet = allotment_wkbk.worksheet_by_title("New Allotment History")

allotment_history_data = pd.DataFrame(allotment_history_sheet.get_all_records())

allotment_history_data = allotment_history_data[['Car + Count','Car Number','Allotment Date','ETM','Return Date']]

allotment_history_data['Return Date'] = allotment_history_data['Return Date'].replace(r'^\s*$', today, regex=True)

allotment_history_data = allotment_history_data.replace(r'^\s*$', np.nan, regex=True)

allotment_history_data = allotment_history_data[allotment_history_data['ETM'].str.contains('ET',na=False)]

allotment_history_data['Allotment Date'] = pd.to_datetime(allotment_history_data['Allotment Date'], dayfirst = True)

allotment_history_data['Return Date'] = pd.to_datetime(allotment_history_data['Return Date'], dayfirst = True)


allotment_history_data['Allotment Date Dummy'] = allotment_history_data['Allotment Date'].dt.date

allotment_history_data['Return Date Dummy'] = allotment_history_data['Return Date'].dt.date

allotment_history_data = allotment_history_data.loc[~(allotment_history_data['Allotment Date Dummy'] == allotment_history_data['Return Date Dummy'])]

adjustment_given_allotment_history = allotment_history_data[['Car Number','Return Date Dummy','ETM']]

adjustment_taken_allotment_history = allotment_history_data[['Car + Count','ETM','Allotment Date Dummy','Allotment Date']]

audit_data = allotment_wkbk.worksheet_by_title("Audit")

allotment_audit_data = pd.DataFrame(audit_data.get_all_records())

allotment_audit_data = allotment_audit_data[['Car + Count','Car Number','CNG - How many bars','Fuel indicator Petrol','Adjustment Amount','Timestamp','DM']]

allotment_audit_data = allotment_audit_data.replace(r'^\s*$', np.nan, regex=True)

allotment_audit_data = allotment_audit_data[allotment_audit_data['Fuel indicator Petrol'].notna()]

allotment_audit_data['Timestamp'] = pd.to_datetime(allotment_audit_data['Timestamp'], dayfirst = True)

allotment_audit_data['start_date'] = pd.Timestamp('07-06-2022')

allotment_audit_data = allotment_audit_data.loc[(allotment_audit_data['Timestamp'] > allotment_audit_data['start_date'])]

allotment_audit_data = allotment_audit_data.loc[~(allotment_audit_data['Adjustment Amount'] == 0)]

allotment_audit_data['Return Date Dummy'] = allotment_audit_data['Timestamp'].dt.date

allotment_audit_data.drop(['start_date'],axis = 1, inplace = True)


adjustment_given = pd.merge(allotment_audit_data,adjustment_given_allotment_history,on=['Return Date Dummy','Car Number'],how='inner')

adjustment_given.rename(columns={"Return Date Dummy":"Bill Date"},inplace = True)

adjustment_given.drop(['Car + Count'],axis = 1, inplace = True)

adjustment_given['start_date'] = pd.Timestamp('08-02-2022')

adjustment_given = adjustment_given.loc[(adjustment_given['Timestamp'] > adjustment_given['start_date'])]

adjustment_given.drop(['start_date'],axis = 1, inplace = True)




allotment_audit_data.drop(['Return Date Dummy','Timestamp'],axis = 1, inplace = True)

adjustment_taken = pd.merge(allotment_audit_data,adjustment_taken_allotment_history,on='Car + Count',how='inner')
adjustment_taken['Adjustment Amount New'] = adjustment_taken['Adjustment Amount'] * -1

adjustment_taken.drop(['Car + Count','Adjustment Amount'],axis = 1, inplace = True)

adjustment_taken.rename(columns={"Allotment Date Dummy":"Bill Date","Allotment Date":"Timestamp","Adjustment Amount New":"Adjustment Amount"},inplace = True)




print(adjustment_given.head())
print(adjustment_taken.head())


adjustment_final = pd.concat([adjustment_taken, adjustment_given], ignore_index=True)


adjustment_final['Name of DM'] = 'Allotment'
adjustment_final['Pilot Name'] = ''
adjustment_final['Adjustment Reason'] = 'Excess Petrol/CNG' 
adjustment_final['Adjustment Comment'] = "CNG:" + adjustment_final['CNG - How many bars'].astype(str) + " / Petrol: " + adjustment_final['Fuel indicator Petrol'].astype(str)
adjustment_final.drop(['CNG - How many bars','Fuel indicator Petrol'],axis = 1, inplace = True)

adjustment_final.sort_values(by=['Timestamp'], ascending=True, inplace=True)

adjustment_final


adjustment_gs = gc.open("testing")

adjustment_sheet = adjustment_gs.worksheet_by_title("test")


adjustment_sheet.set_dataframe(adjustment_final, 'A1')

     Car Number CNG - How many bars Fuel indicator Petrol Adjustment Amount  \
611  MH01DR2735                   1                     0                75   
612  MH01DR3103                   2                     0               105   
613  MH01DR7984                   3                     0               135   
614  MH01DR5211                   0                     1               550   
615  MH01DR8162                   2                     0               105   

              Timestamp               DM   Bill Date       ETM  
611 2022-08-02 10:20:39  Terrific Tigers  2022-08-02  ETM17415  
612 2022-08-02 11:03:15   Black Panthers  2022-08-02  ETM09977  
613 2022-08-02 11:40:11   Silent Killers  2022-08-02  ETM20401  
614 2022-08-02 11:41:56  Terrific Tigers  2022-08-02  ETM21068  
615 2022-08-02 12:17:18  Terrific Tigers  2022-08-02  ETM20841  
Empty DataFrame
Columns: [Car Number, CNG - How many bars, Fuel indicator Petrol, DM, ETM, Bill Date, Timestamp, Adjustment Amount]
Ind

In [4]:
adjustment_gs = gc.open("testing")

adjustment_sheet = adjustment_gs.worksheet_by_title("test")


adjustment_sheet.set_dataframe(adjustment_final, 'A1')

In [1]:
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import DataFrame
import datetime as dt
from datetime import datetime, timedelta,date
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

# try:
#     gc = pygsheets.authorize(service_file='/home/karan/config/car-master-sheet.json')
gc = pygsheets.authorize(service_file='car-master-sheet.json')


# gc = pygsheets.authorize(service_file='/home/ubuntu/scripts/client_secret.json')

today = date.today() + timedelta(days = 1)

allotment_wkbk = gc.open("Allotment Status Report")


allotment_history_sheet = allotment_wkbk.worksheet_by_title("New Allotment History")

allotment_history_data = pd.DataFrame(allotment_history_sheet.get_all_records())

allotment_history_data = allotment_history_data[['Car + Count','Car Number','Allotment Date','ETM','Return Date']]

allotment_history_data['Return Date'] = allotment_history_data['Return Date'].replace(r'^\s*$', today, regex=True)

allotment_history_data = allotment_history_data.replace(r'^\s*$', np.nan, regex=True)

allotment_history_data = allotment_history_data[allotment_history_data['ETM'].str.contains('ET',na=False)]

allotment_history_data['Allotment Date'] = pd.to_datetime(allotment_history_data['Allotment Date'], dayfirst = True)

allotment_history_data['Return Date'] = pd.to_datetime(allotment_history_data['Return Date'], dayfirst = True)

allotment_history_data

,Car + Count,Car Number,Allotment Date,ETM,Return Date
2,MH01CV21711,MH01CV2171,2021-04-07 00:00:00,ETM05532,2022-01-03 15:19:06
5,MH01DR18681,MH01DR1868,2021-04-14 00:00:00,ETM07507,2021-10-25 14:04:00
7,MH01DR18531,MH01DR1853,2021-04-16 00:00:00,ETM06194,2022-05-23 14:48:53
11,MH01DR18671,MH01DR1867,2021-04-21 00:00:00,ETM08098,2021-11-13 17:34:00
15,MH01DR18651,MH01DR1865,2021-04-26 00:00:00,ETM07741,2021-12-06 11:10:38
...,...,...,...,...,...
12770,MH01DR89072,MH01DR8907,2022-08-08 15:41:15,ETM24450,2022-08-09 00:00:00
12777,MH01DR43387,MH01DR4338,2022-08-08 15:20:13,ETM10875,2022-08-09 00:00:00
12784,MH01DR77908,MH01DR7790,2022-08-08 16:10:05,ETM24374,2022-08-09 00:00:00
12786,MH01DR90044,MH01DR9004,2022-08-08 16:07:30,ETM13090,2022-08-09 00:00:00


In [6]:
allotment_history_data['Allotment Date Dummy'] = allotment_history_data['Allotment Date'].dt.date

allotment_history_data['Return Date Dummy'] = allotment_history_data['Return Date'].dt.date

allotment_history_data = allotment_history_data.loc[~(allotment_history_data['Allotment Date Dummy'] == allotment_history_data['Return Date Dummy'])]

adjustment_given_allotment_history = allotment_history_data[['Car Number','Return Date Dummy','ETM']]

adjustment_taken_allotment_history = allotment_history_data[['Car + Count','ETM','Allotment Date Dummy','Allotment Date']]

audit_data = allotment_wkbk.worksheet_by_title("Audit")

allotment_audit_data = pd.DataFrame(audit_data.get_all_records())

allotment_audit_data = allotment_audit_data[['Car + Count','Car Number','CNG - How many bars','Fuel indicator Petrol','Adjustment Amount','Timestamp','DM']]

allotment_audit_data = allotment_audit_data.replace(r'^\s*$', np.nan, regex=True)

allotment_audit_data = allotment_audit_data[allotment_audit_data['Fuel indicator Petrol'].notna()]

allotment_audit_data['Timestamp'] = pd.to_datetime(allotment_audit_data['Timestamp'], dayfirst = True)

allotment_audit_data['start_date'] = pd.Timestamp('07-06-2022')

allotment_audit_data = allotment_audit_data.loc[(allotment_audit_data['Timestamp'] > allotment_audit_data['start_date'])]

allotment_audit_data = allotment_audit_data.loc[~(allotment_audit_data['Adjustment Amount'] == 0)]

allotment_audit_data['Return Date Dummy'] = allotment_audit_data['Timestamp'].dt.date

allotment_audit_data.drop(['start_date'],axis = 1, inplace = True)

allotment_audit_data


,Car + Count,Car Number,Allotment Date,ETM,Return Date,Allotment Date Dummy,Return Date Dummy
2,MH01CV21711,MH01CV2171,2021-04-07 00:00:00,ETM05532,2022-01-03 15:19:06,2021-04-07,2022-01-03
5,MH01DR18681,MH01DR1868,2021-04-14 00:00:00,ETM07507,2021-10-25 14:04:00,2021-04-14,2021-10-25
7,MH01DR18531,MH01DR1853,2021-04-16 00:00:00,ETM06194,2022-05-23 14:48:53,2021-04-16,2022-05-23
11,MH01DR18671,MH01DR1867,2021-04-21 00:00:00,ETM08098,2021-11-13 17:34:00,2021-04-21,2021-11-13
15,MH01DR18651,MH01DR1865,2021-04-26 00:00:00,ETM07741,2021-12-06 11:10:38,2021-04-26,2021-12-06
...,...,...,...,...,...,...,...
12770,MH01DR89072,MH01DR8907,2022-08-08 15:41:15,ETM24450,2022-08-09 00:00:00,2022-08-08,2022-08-09
12777,MH01DR43387,MH01DR4338,2022-08-08 15:20:13,ETM10875,2022-08-09 00:00:00,2022-08-08,2022-08-09
12784,MH01DR77908,MH01DR7790,2022-08-08 16:10:05,ETM24374,2022-08-09 00:00:00,2022-08-08,2022-08-09
12786,MH01DR90044,MH01DR9004,2022-08-08 16:07:30,ETM13090,2022-08-09 00:00:00,2022-08-08,2022-08-09


In [58]:
pd.Timestamp('07-06-2022')

Timestamp('2022-07-06 00:00:00')

# pune

In [29]:
# import os,sys;sys.path.append(os.path.dirname(os.path.realpath('mail')))
# from mail.send_mail import sendMail
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import DataFrame
import datetime as dt
from datetime import datetime, timedelta,date
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

# try:
gc = pygsheets.authorize(service_file='car-master-sheet.json')

# gc = pygsheets.authorize(service_file='/home/ubuntu/scripts/client_secret.json')

today = date.today() + timedelta(days = 1)

allotment_wkbk = gc.open("Allotment Status Report - Pune")


allotment_history_sheet = allotment_wkbk.worksheet_by_title("New Allotment History")

allotment_history_data = pd.DataFrame(allotment_history_sheet.get_all_records())

allotment_history_data = allotment_history_data[['Car + Count','Car Number','Allotment Date','ETM','Return Date']]

allotment_history_data['Return Date'] = allotment_history_data['Return Date'].replace(r'^\s*$', today, regex=True)

allotment_history_data = allotment_history_data.replace(r'^\s*$', np.nan, regex=True)

allotment_history_data = allotment_history_data[allotment_history_data['ETM'].str.contains('ET',na=False)]

allotment_history_data['Allotment Date'] = pd.to_datetime(allotment_history_data['Allotment Date'], dayfirst = True)

allotment_history_data['Return Date'] = pd.to_datetime(allotment_history_data['Return Date'], dayfirst = True)
allotment_history_data

allotment_history_data['Allotment Date Dummy'] = allotment_history_data['Allotment Date'].dt.date

allotment_history_data['Return Date Dummy'] = allotment_history_data['Return Date'].dt.date

allotment_history_data = allotment_history_data.loc[~(allotment_history_data['Allotment Date Dummy'] == allotment_history_data['Return Date Dummy'])]

adjustment_given_allotment_history = allotment_history_data[['Car Number','Return Date Dummy','ETM']]

adjustment_taken_allotment_history = allotment_history_data[['Car + Count','ETM','Allotment Date Dummy','Allotment Date']]

audit_data = allotment_wkbk.worksheet_by_title("Audit")

allotment_audit_data = pd.DataFrame(audit_data.get_all_records())

allotment_audit_data = allotment_audit_data[['Car + Count','Car Number','CNG - How many bars','Fuel indicator Petrol','Adjustment Amount','Timestamp']]

allotment_audit_data = allotment_audit_data.replace(r'^\s*$', np.nan, regex=True)

allotment_audit_data = allotment_audit_data[allotment_audit_data['Fuel indicator Petrol'].notna()]

allotment_audit_data['Timestamp'] = pd.to_datetime(allotment_audit_data['Timestamp'], dayfirst = True)

allotment_audit_data['start_date'] = pd.Timestamp('08-02-2022')

allotment_audit_data = allotment_audit_data.loc[(allotment_audit_data['Timestamp'] > allotment_audit_data['start_date'])]

allotment_audit_data = allotment_audit_data.loc[~(allotment_audit_data['Adjustment Amount'] == 0)]

allotment_audit_data['Return Date Dummy'] = allotment_audit_data['Timestamp'].dt.date

allotment_audit_data.drop(['start_date'],axis = 1, inplace = True)

allotment_audit_data








adjustment_given = pd.merge(allotment_audit_data,adjustment_given_allotment_history,on=['Return Date Dummy','Car Number'],how='inner')

adjustment_given.rename(columns={"Return Date Dummy":"Bill Date"},inplace = True)

adjustment_given.drop(['Car + Count'],axis = 1, inplace = True)

adjustment_given['start_date'] = pd.Timestamp('08-02-2022')

adjustment_given = adjustment_given.loc[(adjustment_given['Timestamp'] > adjustment_given['start_date'])]

adjustment_given.drop(['start_date'],axis = 1, inplace = True)
adjustment_given

allotment_audit_data.drop(['Return Date Dummy','Timestamp'],axis = 1, inplace = True)

adjustment_taken = pd.merge(allotment_audit_data,adjustment_taken_allotment_history,on='Car + Count',how='inner')
adjustment_taken['Adjustment Amount New'] = adjustment_taken['Adjustment Amount'] * -1

adjustment_taken.drop(['Car + Count','Adjustment Amount'],axis = 1, inplace = True)

adjustment_taken.rename(columns={"Allotment Date Dummy":"Bill Date","Allotment Date":"Timestamp","Adjustment Amount New":"Adjustment Amount"},inplace = True)




print(adjustment_given.head())
print(adjustment_taken.head())


adjustment_final = pd.concat([adjustment_taken, adjustment_given], ignore_index=True)


adjustment_final['Name of DM'] = 'Allotment'
adjustment_final['Pilot Name'] = ''
adjustment_final['Adjustment Reason'] = 'Excess Petrol/CNG' 
adjustment_final['Adjustment Comment'] = "CNG:" + adjustment_final['CNG - How many bars'].astype(str) + " / Petrol: " + adjustment_final['Fuel indicator Petrol'].astype(str)
adjustment_final.drop(['CNG - How many bars','Fuel indicator Petrol'],axis = 1, inplace = True)

adjustment_final.sort_values(by=['Timestamp'], ascending=True, inplace=True)

adjustment_final['Car Number']=adjustment_final['Car Number'].str.replace(' ','')

car_no_master_list = gc.open("Car No Master list")
car_no_master_list_history_tab =car_no_master_list.worksheet_by_title("History")
car_no_master_list_history_tab_data = pd.DataFrame(car_no_master_list_history_tab.get_all_records())
car_no_master_list_history_tab_data['Car No']=car_no_master_list_history_tab_data['Car No'].str.replace(' ','')
car_no_master_list_history_tab_data['Date']=pd.to_datetime(car_no_master_list_history_tab_data['Date'], errors='coerce',format='%d-%b-%Y')
car_no_master_list_history_tab_data['End']=pd.to_datetime(car_no_master_list_history_tab_data['End'], errors='coerce',format='%d/%m/%Y')
car_no_master_list_history_tab_data.rename({'Car No':'Car Number'},axis=1,inplace=True)
car_no_master_dm_name=adjustment_final.merge(car_no_master_list_history_tab_data, on='Car Number',how='left')
car_no_master_dm_name_df = car_no_master_dm_name[(car_no_master_dm_name['Date'] <= car_no_master_dm_name['Timestamp']) & (car_no_master_dm_name['Timestamp'] <= car_no_master_dm_name['End'])]
car_no_master_dm_name_df=car_no_master_dm_name_df[['Car Number','DM','ETM','Bill Date','Timestamp','Adjustment Amount','Name of DM','Pilot Name','Adjustment Reason','Adjustment Comment']]
car_no_master_dm_name_df


adjustment_gs = gc.open("Pune Adjustment Sheet")

adjustment_sheet = adjustment_gs.worksheet_by_title("New Allotment Fuel-test")

adjustment_sheet.clear(start='A',end='J')

adjustment_sheet.set_dataframe(car_no_master_dm_name_df, 'A1')

# except Exception:
#     sendMail([
#     'abusalameverestfleet@gmail.com',
#     'avinash.everestfleet@gmail.com',
#     'rutvijaevrestfleet@gmail.com',
#     'sameer.everestfleet@gmail.com',
#     'Karan.saraogi@everestfleet.com',
#     'vijugode@gmail.com'
#     ])

   Car Number CNG - How many bars Fuel indicator Petrol  Adjustment Amount  \
0  MH01DR8598                   1                     0                 75   
1  MH01DR8602                   4                     0                165   
2  MH01DR8167                   7                     0                255   
3  MH01DR8251                   4                     0                165   
4  MH01DR8151                   2                     2                985   

            Timestamp   Bill Date       ETM  
0 2022-08-02 10:48:25  2022-08-02  ETP00212  
1 2022-08-02 10:54:25  2022-08-02  ETP00054  
2 2022-08-02 11:07:00  2022-08-02  ETP00111  
3 2022-08-02 11:15:58  2022-08-02  ETP00097  
4 2022-08-02 11:26:52  2022-08-02  ETP00304  
   Car Number CNG - How many bars Fuel indicator Petrol       ETM   Bill Date  \
0  MH12TV9245                   3                     0  ETP00169  2022-08-03   
1  MH01DR8133                   4                     0  ETP00308  2022-08-03   
2  MH01DR844

In [74]:
adjustment_final['Car Number']=adjustment_final['Car Number'].str.replace(' ','')
adjustment_final['Bill Date']=pd.to_datetime(adjustment_final['Bill Date'], errors='coerce',format='%Y-%m-%d')


Car Number                    object
ETM                           object
Bill Date             datetime64[ns]
Timestamp             datetime64[ns]
Adjustment Amount             object
Name of DM                    object
Pilot Name                    object
Adjustment Reason             object
Adjustment Comment            object
dtype: object

In [81]:
car_no_master_list = gc.open("Car No Master list")
car_no_master_list_history_tab =car_no_master_list.worksheet_by_title("History")
car_no_master_list_history_tab_data = pd.DataFrame(car_no_master_list_history_tab.get_all_records())
car_no_master_list_history_tab_data['Car No']=car_no_master_list_history_tab_data['Car No'].str.replace(' ','')
car_no_master_list_history_tab_data['Date']=pd.to_datetime(car_no_master_list_history_tab_data['Date'], errors='coerce',format='%d-%b-%Y')
car_no_master_list_history_tab_data['End']=pd.to_datetime(car_no_master_list_history_tab_data['End'], errors='coerce',format='%d/%m/%Y')
car_no_master_list_history_tab_data.rename({'Car No':'Car Number'},axis=1,inplace=True)
car_no_master_dm_name=adjustment_final.merge(car_no_master_list_history_tab_data, on='Car Number',how='left')
car_no_master_dm_name_df = car_no_master_dm_name[(car_no_master_dm_name['Date'] <= car_no_master_dm_name['Timestamp']) & (car_no_master_dm_name['Timestamp'] <= car_no_master_dm_name['End'])]
car_no_master_dm_name_df=car_no_master_dm_name_df[['Car Number','DM','ETM','Bill Date','Timestamp','Adjustment Amount','Name of DM','Pilot Name','Adjustment Reason','Adjustment Comment']]
car_no_master_dm_name_df.shape

(24, 10)

# bangalore

In [96]:
# import os,sys;sys.path.append(os.path.dirname(os.path.realpath('mail')))
# from mail.send_mail import sendMail
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import DataFrame
import datetime as dt
from datetime import datetime, timedelta,date
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

# try:
#     gc = pygsheets.authorize(service_file='/home/karan/config/car-master-sheet.json')
gc = pygsheets.authorize(service_file='car-master-sheet.json')
# gc = pygsheets.authorize(service_file='/home/ubuntu/scripts/client_secret.json')

today = date.today() + timedelta(days = 1)

allotment_wkbk = gc.open("Allotment Status Report - Bangalore -2")


allotment_history_sheet = allotment_wkbk.worksheet_by_title("New Allotment History")

allotment_history_data = pd.DataFrame(allotment_history_sheet.get_all_records())

allotment_history_data = allotment_history_data[['Car + Count','Car Number','Allotment Date','ETM','Return Date']]

allotment_history_data['Return Date'] = allotment_history_data['Return Date'].replace(r'^\s*$', today, regex=True)

allotment_history_data = allotment_history_data.replace(r'^\s*$', np.nan, regex=True)

allotment_history_data = allotment_history_data[allotment_history_data['ETM'].str.contains('ET',na=False)]

allotment_history_data['Allotment Date'] = pd.to_datetime(allotment_history_data['Allotment Date'], dayfirst = True)

allotment_history_data['Return Date'] = pd.to_datetime(allotment_history_data['Return Date'], dayfirst = True)
allotment_history_data

allotment_history_data['Allotment Date Dummy'] = allotment_history_data['Allotment Date'].dt.date

allotment_history_data['Return Date Dummy'] = allotment_history_data['Return Date'].dt.date

allotment_history_data = allotment_history_data.loc[~(allotment_history_data['Allotment Date Dummy'] == allotment_history_data['Return Date Dummy'])]

adjustment_given_allotment_history = allotment_history_data[['Car Number','Return Date Dummy','ETM']]

adjustment_taken_allotment_history = allotment_history_data[['Car + Count','ETM','Allotment Date Dummy','Allotment Date']]

audit_data = allotment_wkbk.worksheet_by_title("Audit")

allotment_audit_data = pd.DataFrame(audit_data.get_all_records())

allotment_audit_data = allotment_audit_data[['Car + Count','Car Number','CNG - How many bars','Fuel indicator Petrol','Adjustment Amount','Timestamp']]

allotment_audit_data = allotment_audit_data.replace(r'^\s*$', np.nan, regex=True)

allotment_audit_data = allotment_audit_data[allotment_audit_data['Fuel indicator Petrol'].notna()]

allotment_audit_data['Timestamp'] = pd.to_datetime(allotment_audit_data['Timestamp'], dayfirst = True)

allotment_audit_data['start_date'] = pd.Timestamp('08-05-2022')

allotment_audit_data = allotment_audit_data.loc[(allotment_audit_data['Timestamp'] > allotment_audit_data['start_date'])]

allotment_audit_data = allotment_audit_data.loc[~(allotment_audit_data['Adjustment Amount'] == 0)]

allotment_audit_data['Return Date Dummy'] = allotment_audit_data['Timestamp'].dt.date

allotment_audit_data.drop(['start_date'],axis = 1, inplace = True)
allotment_audit_data

adjustment_given = pd.merge(allotment_audit_data,adjustment_given_allotment_history,on=['Return Date Dummy','Car Number'],how='inner')

adjustment_given.rename(columns={"Return Date Dummy":"Bill Date"},inplace = True)

adjustment_given.drop(['Car + Count'],axis = 1, inplace = True)

adjustment_given['start_date'] = pd.Timestamp('08-05-2022')

adjustment_given = adjustment_given.loc[(adjustment_given['Timestamp'] > adjustment_given['start_date'])]

adjustment_given.drop(['start_date'],axis = 1, inplace = True)
adjustment_given

allotment_audit_data.drop(['Return Date Dummy','Timestamp'],axis = 1, inplace = True)

adjustment_taken = pd.merge(allotment_audit_data,adjustment_taken_allotment_history,on='Car + Count',how='inner')
adjustment_taken['Adjustment Amount New'] = adjustment_taken['Adjustment Amount'] * -1

adjustment_taken.drop(['Car + Count','Adjustment Amount'],axis = 1, inplace = True)

adjustment_taken.rename(columns={"Allotment Date Dummy":"Bill Date","Allotment Date":"Timestamp","Adjustment Amount New":"Adjustment Amount"},inplace = True)

print(adjustment_given.head())
print(adjustment_taken.head())

adjustment_final = pd.concat([adjustment_taken, adjustment_given], ignore_index=True)


adjustment_final['Name of DM'] = 'Allotment'
adjustment_final['Pilot Name'] = ''
adjustment_final['Adjustment Reason'] = 'Excess Petrol/CNG' 
adjustment_final['Adjustment Comment'] = "CNG:" + adjustment_final['CNG - How many bars'].astype(str) + " / Petrol: " + adjustment_final['Fuel indicator Petrol'].astype(str)
adjustment_final.drop(['CNG - How many bars','Fuel indicator Petrol'],axis = 1, inplace = True)

adjustment_final.sort_values(by=['Timestamp'], ascending=True, inplace=True)


car_no_master_list = gc.open("Car Master No List Bangalore")
car_no_master_list_history_tab =car_no_master_list.worksheet_by_title("History")
car_no_master_list_history_tab_data = pd.DataFrame(car_no_master_list_history_tab.get_all_records())
car_no_master_list_history_tab_data['Car No']=car_no_master_list_history_tab_data['Car No'].str.replace(' ','')
car_no_master_list_history_tab_data['Date']=pd.to_datetime(car_no_master_list_history_tab_data['Date'], errors='coerce',format='%d-%b-%Y')
car_no_master_list_history_tab_data['End']=pd.to_datetime(car_no_master_list_history_tab_data['End'], errors='coerce',format='%d/%m/%Y')
car_no_master_list_history_tab_data.rename({'Car No':'Car Number'},axis=1,inplace=True)
car_no_master_dm_name=adjustment_final.merge(car_no_master_list_history_tab_data, on='Car Number',how='left')
car_no_master_dm_name_df = car_no_master_dm_name[(car_no_master_dm_name['Date'] <= car_no_master_dm_name['Timestamp']) & (car_no_master_dm_name['Timestamp'] <= car_no_master_dm_name['End'])]
car_no_master_dm_name_df=car_no_master_dm_name_df[['Car Number','DM','ETM','Bill Date','Timestamp','Adjustment Amount','Name of DM','Pilot Name','Adjustment Reason','Adjustment Comment']]
car_no_master_dm_name_df


adjustment_gs = gc.open("Rental Security Deposit (Responses)")

adjustment_sheet = adjustment_gs.worksheet_by_title("New Adjustment Fuel")


adjustment_sheet.set_dataframe(car_no_master_dm_name_df, 'A1')




# except Exception:
#     sendMail([
#     'abusalameverestfleet@gmail.com',
#     'avinash.everestfleet@gmail.com',
#     'rutvijaevrestfleet@gmail.com',
#     'sameer.everestfleet@gmail.com',
#     'Karan.saraogi@everestfleet.com',
#     'vijugode@gmail.com'
#     ])


   Car Number CNG - How many bars Fuel indicator Petrol Adjustment Amount  \
0  KA05AL1776                 NaN                     1               500   
1  KA05AL2075                 NaN                     2               800   
2  KA05AK7892                 NaN                     2               800   
3  KA05AL2879                 NaN                     2               800   
4  KA05AK8494                 NaN                     1               500   

            Timestamp   Bill Date       ETM  
0 2022-08-05 10:34:21  2022-08-05  ETB04239  
1 2022-08-05 10:45:38  2022-08-05  ETB03490  
2 2022-08-05 16:15:21  2022-08-05  ETB01407  
3 2022-08-05 17:06:55  2022-08-05  ETB04314  
4 2022-08-05 17:40:58  2022-08-05  ETB03115  
   Car Number CNG - How many bars Fuel indicator Petrol       ETM   Bill Date  \
0  KA05AL1776                 NaN                     1  ETB04041  2022-08-05   
1  KA05AL2075                 NaN                     2  ETB02008  2022-08-05   
2  KA05AL2879     

In [42]:
# import os,sys;sys.path.append(os.path.dirname(os.path.realpath('mail')))
# from mail.send_mail import sendMail
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import DataFrame
import datetime as dt
from datetime import datetime, timedelta,date
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

# try:
#     gc = pygsheets.authorize(service_file='/home/karan/config/car-master-sheet.json')
gc = pygsheets.authorize(service_file='car-master-sheet.json')


# gc = pygsheets.authorize(service_file='/home/ubuntu/scripts/client_secret.json')

today = date.today() + timedelta(days = 1)

allotment_wkbk = gc.open("Allotment Status Report - Bangalore -2")


allotment_history_sheet = allotment_wkbk.worksheet_by_title("New Allotment History")

allotment_history_data = pd.DataFrame(allotment_history_sheet.get_all_records())

allotment_history_data = allotment_history_data[['Car + Count','Car Number','Allotment Date','ETM','Return Date']]

allotment_history_data['Return Date'] = allotment_history_data['Return Date'].replace(r'^\s*$', today, regex=True)

allotment_history_data = allotment_history_data.replace(r'^\s*$', np.nan, regex=True)

allotment_history_data = allotment_history_data[allotment_history_data['ETM'].str.contains('ET',na=False)]

allotment_history_data['Allotment Date'] = pd.to_datetime(allotment_history_data['Allotment Date'], dayfirst = True)

allotment_history_data['Return Date'] = pd.to_datetime(allotment_history_data['Return Date'], dayfirst = True)
allotment_history_data

,Car + Count,Car Number,Allotment Date,ETM,Return Date
0,KA05AK66711,KA05AK6671,2021-12-20 19:13:00,ETB01137,2022-07-31
1,KA53AA0423 1,KA53AA0423,2022-01-17 12:18:00,ETB01488,2022-08-06
2,KA05AK66721,KA05AK6672,2022-01-18 18:45:00,ETB01727,2022-06-20
3,KA05AK66811,KA05AK6681,2022-01-24 10:54:00,ETB01388,2022-04-25
4,KA05AK78451,KA05AK7845,2022-02-19 18:17:00,ETB00750,2022-05-16
...,...,...,...,...,...
2661,KA05AL20482,KA05AL2048,2022-08-04 20:12:11,ETB04371,2022-08-06
2664,KA05AK94089,KA05AK9408,2022-08-04 18:25:41,ETB03443,2022-08-06
2670,KA05AK84954,KA05AK8495,2022-08-04 19:07:45,ETB02936,2022-08-06
2673,KA05AL05215,KA05AL0521,2022-08-05 13:03:53,ETB03679,2022-08-06


In [43]:
allotment_history_data['Allotment Date Dummy'] = allotment_history_data['Allotment Date'].dt.date

allotment_history_data['Return Date Dummy'] = allotment_history_data['Return Date'].dt.date

allotment_history_data = allotment_history_data.loc[~(allotment_history_data['Allotment Date Dummy'] == allotment_history_data['Return Date Dummy'])]

adjustment_given_allotment_history = allotment_history_data[['Car Number','Return Date Dummy','ETM']]

adjustment_taken_allotment_history = allotment_history_data[['Car + Count','ETM','Allotment Date Dummy','Allotment Date']]

audit_data = allotment_wkbk.worksheet_by_title("Audit")

allotment_audit_data = pd.DataFrame(audit_data.get_all_records())

allotment_audit_data = allotment_audit_data[['Car + Count','Car Number','CNG - How many bars','Fuel indicator Petrol','Adjustment Amount','Timestamp','DM']]

allotment_audit_data = allotment_audit_data.replace(r'^\s*$', np.nan, regex=True)

allotment_audit_data = allotment_audit_data[allotment_audit_data['Fuel indicator Petrol'].notna()]

allotment_audit_data['Timestamp'] = pd.to_datetime(allotment_audit_data['Timestamp'], dayfirst = True)

allotment_audit_data
allotment_audit_data['start_date'] = pd.Timestamp('08-05-2022')

allotment_audit_data = allotment_audit_data.loc[(allotment_audit_data['Timestamp'] > allotment_audit_data['start_date'])]

allotment_audit_data = allotment_audit_data.loc[~(allotment_audit_data['Adjustment Amount'] == 0)]

allotment_audit_data['Return Date Dummy'] = allotment_audit_data['Timestamp'].dt.date

allotment_audit_data.drop(['start_date'],axis = 1, inplace = True)
allotment_audit_data


,Car + Count,Car Number,CNG - How many bars,Fuel indicator Petrol,Adjustment Amount,Timestamp,DM,Return Date Dummy
2776,KA05AL17763,KA05AL1776,NaN,1,500,2022-08-05 10:34:21,Titans,2022-08-05
2777,KA05AL20754,KA05AL2075,NaN,2,800,2022-08-05 10:45:38,Titans,2022-08-05


In [44]:
adjustment_given = pd.merge(allotment_audit_data,adjustment_given_allotment_history,on=['Return Date Dummy','Car Number'],how='inner')

adjustment_given.rename(columns={"Return Date Dummy":"Bill Date"},inplace = True)

adjustment_given.drop(['Car + Count'],axis = 1, inplace = True)

adjustment_given['start_date'] = pd.Timestamp('08-05-2022')

adjustment_given = adjustment_given.loc[(adjustment_given['Timestamp'] > adjustment_given['start_date'])]

adjustment_given.drop(['start_date'],axis = 1, inplace = True)
adjustment_given

allotment_audit_data.drop(['Return Date Dummy','Timestamp'],axis = 1, inplace = True)

adjustment_taken = pd.merge(allotment_audit_data,adjustment_taken_allotment_history,on='Car + Count',how='inner')
adjustment_taken['Adjustment Amount New'] = adjustment_taken['Adjustment Amount'] * -1

adjustment_taken.drop(['Car + Count','Adjustment Amount'],axis = 1, inplace = True)

adjustment_taken.rename(columns={"Allotment Date Dummy":"Bill Date","Allotment Date":"Timestamp","Adjustment Amount New":"Adjustment Amount"},inplace = True)

print(adjustment_given.head())
print(adjustment_taken.head())

   Car Number CNG - How many bars Fuel indicator Petrol Adjustment Amount  \
0  KA05AL1776                 NaN                     1               500   
1  KA05AL2075                 NaN                     2               800   

            Timestamp      DM   Bill Date       ETM  
0 2022-08-05 10:34:21  Titans  2022-08-05  ETB04239  
1 2022-08-05 10:45:38  Titans  2022-08-05  ETB03490  
Empty DataFrame
Columns: [Car Number, CNG - How many bars, Fuel indicator Petrol, DM, ETM, Bill Date, Timestamp, Adjustment Amount]
Index: []


In [45]:
adjustment_final = pd.concat([adjustment_taken, adjustment_given], ignore_index=True)


adjustment_final['Name of DM'] = 'Allotment'
adjustment_final['Pilot Name'] = ''
adjustment_final['Adjustment Reason'] = 'Excess Petrol/CNG' 
adjustment_final['Adjustment Comment'] = "CNG:" + adjustment_final['CNG - How many bars'].astype(str) + " / Petrol: " + adjustment_final['Fuel indicator Petrol'].astype(str)
adjustment_final.drop(['CNG - How many bars','Fuel indicator Petrol'],axis = 1, inplace = True)

adjustment_final.sort_values(by=['Timestamp'], ascending=True, inplace=True)

adjustment_final

,Car Number,DM,ETM,Bill Date,Timestamp,Adjustment Amount,Name of DM,Pilot Name,Adjustment Reason,Adjustment Comment
0,KA05AL1776,Titans,ETB04239,2022-08-05,2022-08-05 10:34:21,500,Allotment,,Excess Petrol/CNG,CNG:nan / Petrol: 1
1,KA05AL2075,Titans,ETB03490,2022-08-05,2022-08-05 10:45:38,800,Allotment,,Excess Petrol/CNG,CNG:nan / Petrol: 2


In [ ]:
# adjustment_gs = gc.open("New-Adjustment Sheet")

# adjustment_sheet = adjustment_gs.worksheet_by_title("New Allotment Fuel")


# adjustment_sheet.set_dataframe(adjustment_final, 'A1')